#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계2 : 모델링

## 0.미션

* 다음 세가지 챗봇을 만들고 비교해 봅시다.
* 챗봇1. Word2Vec 임베딩 벡터 기반 머신러닝 분류 모델링
    * Word2Vec 모델을 만들고 임베딩 벡터를 생성합니다.
    * 임베딩 벡터를 이용하여 intent를 분류하는 모델링을 수행합니다.
        * 이때, LightGBM을 추천하지만, 다른 알고리즘을 이용할수 있습니다.
    * 예측된 intent의 답변 중 임의의 하나를 선정하여 출력합니다.
* 챗봇2. 단계별 모델링1
    * 1단계 : type(일상대화 0, 에이블스쿨Q&A 1) 분류 모델 만들기
        * Embedding + LSTM 모델링
    * 2단계 : 사전학습된 Word2Vec 모델을 로딩하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기
* 챗봇3. 단계별 모델링2
    * 1단계 : 챗봇2의 1단계 모델을 그대로 활용
    * 2단계 : FastText 모델 생성하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기

* 챗봇3개에 대해서 몇가지 질문을 입력하고 각각의 답변을 비교해 봅시다.


## 1.환경준비

### (1)라이브러리 설치

#### 1) gensim 3.8.3 설치

In [1]:
#gensim은 자연어 처리를 위한 오픈소스 라이브러리입니다. 토픽 모델링, 워드 임베딩 등 다양한 자연어 처리 기능을 제공
# 현재 4.x 버전이 최신이지만, 3.8.3 버전으로 진행
!pip install gensim==3.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


* [코랩] 위 라이브러리 설치후 런타임 재시작 필요!

#### 2) 형태소 분석을 위한 라이브러리

In [2]:
# mecab 설치를 위한 관련 패키지 설치
!apt-get install curl git
!apt-get install build-essential
!apt-get install cmake
!apt-get install g++
!apt-get install flex
!apt-get install bison
!apt-get install python-dev
!pip install cython
!pip install mecab-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.18).
git is already the newest version (1:2.25.1-1ubuntu3.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Rea

In [3]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu: linux, mac os 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


### (2) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 필요하다고 판단되는 라이브러리를 추가하세요.
import os

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import * 

import tensorflow as tf
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics.pairwise import cosine_similarity



* 형태소 분석을 위한 함수를 제공합니다.

In [5]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        # "kkma":
        tokenizer = Kkma()
        
    return tokenizer

In [6]:
# 형태소 분석을 수행하는 함수

def tokenize(tokenizer_name, original_sent, nouns=False):
    # 미리 정의된 몇 가지 tokenizer 중 하나를 선택
    tokenizer = get_tokenizer(tokenizer_name)

    # tokenizer를 이용하여 original_sent를 토큰화하여 tokenized_sent에 저장하고, 이를 반환합니다.
    sentence = original_sent.replace('\n', '').strip()
    if nouns:       
        # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.nouns(sentence)
    else:
        tokens = tokenizer.morphs(sentence)
    # tokenized_sent = ' '.join(tokens)
    return tokens
    # return tokenized_sent

### (3) 데이터 로딩
* 전처리 단계에서 생성한 데이터들을 로딩합니다.
    * train, test
    * 형태소분석 결과 데이터 : clean_train_questions, clean_test_questions

* 구글 드라이브 연결

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
path = '/content/drive/MyDrive/aivle/miniProject/project6/'

* 저장된 .pkl 파일들을 불러옵니다.
* 불러 온 후에는 shape를 확인해 봅시다.

In [9]:
import joblib
train_data = joblib.load(path+'train.pkl')
test_data = joblib.load(path+'test.pkl')
clean_train_questions = joblib.load(path+'clean_train_questions.pkl')
clean_test_questions = joblib.load(path+'clean_test_questions.pkl')

## 2.챗봇1

* **상세요구사항**
    * Word2Vec을 활용한 LightGBM 모델링(intent 분류)
        * Word2Vec을 이용하여 임베딩벡터 생성하기
            * Word Embedding으로 문장벡터 구하기
        * 임베딩 벡터를 이용하여 ML기반 모델링 수행하기
            * LightGBM 권장(다른 알고리즘을 이용할수 있습니다.)
    * 챗봇 : 모델의 예측결과(intent)에 따라 답변하는 챗봇 만들기
        * 질문을 입력받아, 답변하는 함수 생성

### (1) Word2Vec을 이용하여 임베딩벡터 생성하기
* 'mecab' 형태소 분석기를 이용하여 문장을 tokenize
    * Word2Vec 모델을 만들기 위해서 입력 데이터는 리스트 형태여야 합니다.
    * 그래서 다시 리스트로 저장되도록 토크나이즈 해 봅시다.
* Word Embedding으로 문장벡터를 생성합니다.
    * 먼저 Word2Vec 모델을 만들고, train의 질문들을 문장벡터로 만듭시다.


#### 1) 'mecab' 형태소 분석기를 이용하여 문장을 tokenize

In [10]:
train_data.drop('str_len',axis=1, inplace=True)

In [11]:
test_data.drop('str_len',axis=1, inplace=True)

In [12]:
# train_data['Q'] = train_data['Q'].apply(lambda x:tokenize('mecab',x))
# test_data['Q'] = test_data['Q'].apply(lambda x:tokenize('mecab',x))

In [13]:
train_token = []
for sentence in train_data['Q']:
    tokenized = tokenize('mecab', sentence)
    train_token.append(tokenized)

In [14]:
train_token[:5]

[['떨어뜨려서', '핸드폰', '액정', '나갔', '어'],
 ['액정', '나갔', '어'],
 ['핸드폰', '떨어뜨려서', '고장', '났', '나', '봐'],
 ['노트북', '키보드', '가', '안', '먹히', '네'],
 ['노트북', '을', '떨어뜨려서', '고장', '난', '것', '같', '아', '.']]

In [15]:
test_token = []
for sentence in test_data['Q']:
    tokenized = tokenize('mecab', sentence)
    test_token.append(tokenized)

#### 2) Word Embedding으로 문장벡터 구하기
* Word2Vec
    * 위에서 저장한 입력 데이터를 사용하여 Word2Vec 모델이 생성
    * 모델은 size(단어 벡터의 차원), 
    * window(컨텍스트 창의 크기), 
    * max_vocab_size(고려할 최대 어휘 크기), 
    * min_count(포함할 단어의 최소 빈도)와 같은 특정 하이퍼파라미터로 훈련됩니다.
    * sg : 사용할 훈련 알고리즘 - 1은 skip-gram, 0은 CBOW )

In [16]:
from gensim.models import Word2Vec

# Word2Vec 모델 생성
wv_model = Word2Vec(sentences=train_token, size=100, window=5, min_count=2, sg=0, batch_words=8)

* Word2Vec 모델로부터 데이터를 벡터화하기 위한 함수 생성

In [17]:
# Word2Vec 모델로부터 하나의 문장을 벡터화 시키는 함수
def get_sent_embedding(model, embedding_size, tokenized_words):
    # 임베딩 벡터를 0으로 초기화
    feature_vec = np.zeros((embedding_size,), dtype='float32')
    # 단어 개수 초기화
    n_words = 0
    # 모델 단어 집합 생성
    index2word_set = set(model.wv.index2word)
    # 문장의 단어들을 하나씩 반복
    for word in tokenized_words:
        # 모델 단어 집합에 해당하는 단어일 경우에만
        if word in index2word_set:
            # 단어 개수 1 증가
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model[word])
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [18]:
# 문장벡터 데이터 셋 만들기
def get_dataset(sentences, model, num_features):
    dataset = list()

    # 각 문장을 벡터화해서 리스트에 저장
    for sent in sentences:
        dataset.append(get_sent_embedding(model, num_features, sent))

    # 리스트를 numpy 배열로 변환하여 반환
    sent_embedding_vectors = np.stack(dataset)
    
    return sent_embedding_vectors

* 이제 학습데이터의 Q를 Word2Vec 모델을 사용하여 벡터화 합니다.

In [19]:
# 학습 데이터의 문장들을 Word2Vec 모델을 사용하여 벡터화
train_data_vecs = get_dataset(train_token, wv_model, 100)


In [20]:
len(train_data_vecs)

1107

* 훈련된 Word2Vec 모델을 사용하여 문장 목록에 대한 문장 임베딩을 생성하고 이를 2차원 numpy 배열에 저장합니다. 
* 그런 다음 이러한 임베딩을 다양한 기계 학습 모델의 입력 기능으로 사용할 수 있습니다

### (2) 분류 모델링
* 데이터 분할
    * x, y
        * x : 이전 단계에서 저장된 임베딩벡터(train_data_vecs)
        * y : intent 값들
    * train, val
        * train_test_split 활용
* 머신러닝 모델링
    * lightGBM, RandomForest 등을 활용하여 학습
    * 필요하다면 hyper parameter 튜닝을 시도해도 좋습니다.
* validation set으로 검증해 봅시다.

In [21]:
# X와 y 데이터 분리
x = train_data_vecs
y = train_data['intent']

# Train-Test split
x_train,x_val, y_train,y_val = train_test_split(x,y, test_size=0.2)


* 모델1

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
model = RandomForestClassifier()

In [24]:
model.fit(x_train,y_train)

RandomForestClassifier()

* 모델2

### (3) 챗봇 구축

* **상세요구사항**
    * 챗봇 flow : input 질문 -> 분류 모델로 intent 예측 --> intent에 해당하는 답변 출력
        * 하나의 intent 에는 여러 답변이 있습니다. 이중 한가지를 랜덤하게 선택합니다.

#### 1) 데이터 중 하나에 대해서 테스트

In [25]:
train_data.loc[train_data['intent']==2,'A'].unique()

array(['가장 중요한 거예요.', '가장 중요한 목표네요.'], dtype=object)

In [26]:

y_pred = model.predict(x_val)


In [27]:
# y_pred = model.predict(test_sen)

In [28]:
y_pred[0]
y_val

156      8
3        1
339     26
364     28
794     43
        ..
705     39
1098    53
742     40
562     32
384     29
Name: intent, Length: 222, dtype: int64

#### 2) 챗봇 함수 만들기
* 테스트 코드를 바탕으로 질문을 받아 답변을 하는 함수를 생성합시다.
* 성능이 좋은 모델 사용. 

In [29]:
def get_answer1(question): 




    return 

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 3.챗봇2

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : type을 0과 1로 분류하는 모델 생성(Embedding + LSTM 모델)
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링(LSTM)
- LSTM

#### 1) 데이터 준비
* 학습용 데이터를 만들어 봅시다.
    * 시작 데이터 : clean_train_questions, clean_test_questions
    * 각 토큰에 인덱스를 부여하는 토크나이저를 만들고 적용
        * from tensorflow.keras.preprocessing.text import Tokenizer 를 사용
    * 문장별 길이에 대한 분포를 확인하고 적절하게 정의.

In [30]:
# 각각의 토큰에 인덱스 부여하는 토크나이저 선언
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=35000
# .fit_on_tests 이용하여 토크나이저 만들기
tokenizer = Tokenizer(num_words=max_words, lower=False)
tokenizer.fit_on_texts(clean_train_questions)
word_dic = tokenizer.word_index
print(word_dic)

{'나요': 1, '이': 2, '있': 3, '는': 4, '가': 5, '하': 6, '교육': 7, '지원': 8, '을': 9, '에': 10, '되': 11, '수': 12, '할': 13, '어': 14, '도': 15, '한가요': 16, '가능': 17, '경우': 18, '은': 19, '고': 20, '어떻게': 21, '으로': 22, '면': 23, '를': 24, '나': 25, '중': 26, '로': 27, '없': 28, '서류': 29, '한': 30, '궁금': 31, '노트북': 32, '는데': 33, '해야': 34, '어떤': 35, '합니다': 36, '에서': 37, '졸업': 38, '과': 39, '진행': 40, '의': 41, '다': 42, '수강': 43, '시': 44, '싶': 45, '해': 46, '받': 47, '인': 48, '채용': 49, '과정': 50, '만': 51, '인가요': 52, '연계': 53, '지': 54, '다른': 55, '아': 56, '게': 57, '보': 58, '안': 59, '먹': 60, '주': 61, '대면': 62, '지역': 63, '했': 64, '제공': 65, '자': 66, '시간': 67, '제출': 68, '취업': 69, '증명서': 70, '합격': 71, '좋': 72, '공부': 73, '뭔가요': 74, '추가': 75, '거': 76, '같': 77, '들': 78, '기준': 79, '미취': 80, '업자': 81, '생': 82, '을까요': 83, '비대': 84, '것': 85, '너무': 86, '참여': 87, '일': 88, '와': 89, '해도': 90, '수료': 91, '프리랜서': 92, '재': 93, '검사': 94, '장소': 95, '기간': 96, '실업': 97, '급여': 98, 'KT': 99, '오늘': 100, '기': 101, '뭐': 102, '대학': 103, '테스트': 104, '네

In [31]:
# 전체 토큰의 수 확인
# train_seq = tokenizer.texts_to_sequences(clean_train_questions)
# print(train_seq)
# padded = pad_sequences(train_seq)
# print(padded)


[[369, 112, 475, 476, 14], [475, 476, 14], [112, 369, 477, 478, 25, 170], [32, 695, 5, 59, 696, 105], [32, 9, 369, 477, 479, 85, 77, 56], [152, 5, 697, 698, 6, 54, 153, 699], [152, 5, 700, 701, 24, 702, 20, 3, 370], [371, 303, 480, 25, 170], [371, 303, 5, 703, 2, 372, 14], [371, 303, 5, 59, 268], [112, 480, 25, 170], [112, 59, 268], [32, 59, 268], [113, 704, 4, 373, 25, 170], [113, 41, 705, 59, 11, 4, 88, 374, 170], [706, 23, 59, 11, 33], [215, 15, 481, 6, 237], [215, 15, 482, 6, 237], [215, 69, 6, 237], [215, 71, 6, 237], [215, 4, 483, 69, 13, 76, 114], [238, 171, 15, 4, 483, 154, 9, 707, 13, 76, 114], [708, 119, 24, 375, 20, 45, 14], [215, 15, 481, 6, 20, 482, 6, 57, 269, 76, 114], [113, 5, 270, 6, 4, 88, 9, 484, 20, 45, 14], [113, 5, 72, 56, 6, 4, 88, 19, 120, 485, 484, 56, 709], [172, 376, 304, 377, 378, 155, 14], [377, 2, 42, 378, 155, 14], [486, 20, 305], [194, 487], [379, 380, 173, 64, 14], [379, 380, 76, 77, 56], [271, 140, 306], [271, 2, 86, 140, 306], [174, 175, 100, 86, 307,

In [32]:
# 전체 토큰의 수로 vocab_size 지정
vocab_size = len(word_dic)

# fit_on_texts을 위에서 한번만 해도 되지만, vocab 사이즈를 확인하고 줄이거나 하는 시도를 할 수도 있기에 다시 수행
tokenizer.fit_on_texts(clean_train_questions)
# .texts_to_sequences : 토크나이즈 된 데이터를 가지고 모두 시퀀스로 변환
x_train= tokenizer.texts_to_sequences(clean_train_questions)
x_val =  tokenizer.texts_to_sequences(clean_test_questions)

In [59]:
x_train.shape

(1107, 54)

In [60]:
x_val.shape

(104, 19)

In [61]:
# 각 토큰과 인덱스로 구성된 딕셔너리 생성,
word_dic = tokenizer.word_index

# <PAD> 는 0으로 추가
x_train = pad_sequences(x_train,maxlen=54)
x_val = pad_sequences(x_val,maxlen=54)

* 문장별 토큰수에 대해 탐색적 분석을 수행해 봅시다.

In [34]:
len(x_train[0])

54

* 문장별 토큰이 가장 큰 것이 57개 입니다. 

* 학습 입력을 위한 데이터 크기 맞추기
    * 문장이 짧기 때문에 MAX_SEQUENCE_LENGTH는 정하지 않아도 되지만,
    * 그러나 분포를 보고 적절하게 자릅시다.
    * 그리고 pad_sequences 함수를 이용하여 시퀀스데이터로 변환하기
* y는 train['type'] 와 test['type'] 입니다.

In [55]:
y_train = np.array(train_data['type'].to_list())
y_val = np.array(test_data['type'].to_list())

In [36]:
x_train

array([[  0,   0,   0, ..., 475, 476,  14],
       [  0,   0,   0, ..., 475, 476,  14],
       [  0,   0,   0, ..., 478,  25, 170],
       ...,
       [  0,   0,   0, ...,   2,   3,   1],
       [  0,   0,   0, ..., 158,  31, 432],
       [  0,   0,   0, ...,   2,   3,   1]], dtype=int32)

In [46]:
x_train.shape

(1107, 54)

#### 2) 모델링

* 토크나이징 한 데이터를 입력으로 받아 
* Embedding 레이어와 LSTM 레이어를 결합하여 이진 분류 모델링을 수행합니다.

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Embedding

In [52]:
x_train

array([[  0,   0,   0, ..., 475, 476,  14],
       [  0,   0,   0, ..., 475, 476,  14],
       [  0,   0,   0, ..., 478,  25, 170],
       ...,
       [  0,   0,   0, ...,   2,   3,   1],
       [  0,   0,   0, ..., 158,  31, 432],
       [  0,   0,   0, ...,   2,   3,   1]], dtype=int32)

In [56]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [62]:
x_train.shape,x_val.shape,y_train.shape,y_val.shape

((1107, 54), (104, 54), (1107,), (104,))

In [63]:
max_words=30000
embedding_dim = 128
max_len = 54

tf.keras.backend.clear_session()

model = keras.models.Sequential()

model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(keras.layers.LSTM(16,activation='tanh' ) )
model.add(   keras.layers.Dense(1,activation='sigmoid') )
model.compile(  loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'] )

# 학습

history = model.fit( x_train , y_train  , epochs = 30 , validation_data=(x_val, y_val))

Epoch 1/30
35/35 [==============================] - 7s 135ms/step - loss: 0.5785 - val_loss: 0.4226
Epoch 2/30
35/35 [==============================] - 3s 80ms/step - loss: 0.2207 - val_loss: 0.1540
Epoch 3/30
35/35 [==============================] - 4s 104ms/step - loss: 0.0798 - val_loss: 0.0758
Epoch 4/30
35/35 [==============================] - 5s 156ms/step - loss: 0.0364 - val_loss: 0.0457
Epoch 5/30
35/35 [==============================] - 7s 188ms/step - loss: 0.0208 - val_loss: 0.0314
Epoch 6/30
35/35 [==============================] - 3s 83ms/step - loss: 0.0135 - val_loss: 0.0258
Epoch 7/30
35/35 [==============================] - 3s 78ms/step - loss: 0.0097 - val_loss: 0.0197
Epoch 8/30
35/35 [==============================] - 3s 79ms/step - loss: 0.0073 - val_loss: 0.0173
Epoch 9/30
35/35 [==============================] - 4s 112ms/step - loss: 0.0057 - val_loss: 0.0140
Epoch 10/30
35/35 [==============================] - 3s 79ms/step - loss: 0.0047 - val_loss: 0.0122
Epoc

In [68]:
max(train_data['Q'].str.len())

127

### (2) 사전학습모델(Word2Vec)

* Pre-trained Word2Vec model
    * 이미 다운로드 받아서 제공되었습니다.
        * ko.bin, ko.tsv
    * 참고 사이트: https://github.com/Kyubyong/wordvectors
        * 모델 파일 다운로드 사이트: https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view?resourcekey=0-Dq9yyzwZxAqT3J02qvnFwg
* 사전학습 모델을 로딩하고, 
* train 데이터셋의 질문(Q)을 임베딩벡터로 만들어, 열(Column)로 추가합니다.

#### 1) 모델 로딩
* 사전 학습된 모델을 로딩 : gensim.models.Word2Vec.load()
* 로딩 후 벡터 크기를 조회합시다. .vector_size

In [80]:
import gensim
pre_wv_model = gensim.models.Word2Vec.load(path + 'ko.bin')

In [97]:
# 모델의 벡터크기 조회
pre_wv_model.vector_size

200

#### 2) train 에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

In [122]:
train_data['embedded'] = train_data['Q'].apply(lambda x: get_sent_embedding(pre_wv_model, 200, tokenize('mecab',x)))

### (3) 챗봇 구축
* 아래 절차대로 수행하는 함수 만들기
    * input 질문 
    * 1단계 : 모델을 이용하여 type 0, 1로 분류
    * 2단계 : 
        * train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
        * 코사인 유사도가 가장 높은 Q를 선택
        * 선택한 Q의 intent에 맵핑된 답변 중 하나를 무작위로 선택

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

* 예측을 위한 입력 형태로 변환
    * 학습을 위한 전처리 과정을 test 데이터에도 적용합니다. 

* 1단계 : type 분류

* 2단계 : 질문에 대한 벡터 만들고 코사인 유사도 계산
    * Word2Vec 사전 학습 모델로 부터 벡터 만들기

* train의 질문 벡터들과 유사도 계산
    * Word2Vec 으로 만든 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity






#### 2) 챗봇 함수 만들기
* 위 테스트 결과를 바탕으로 코드를 정리하고 함수로 생성합니다.

In [ ]:
def get_answer2(question): 





    return

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 4.챗봇3

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : 챗봇2의 1단계 모델을 사용합니다.
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링
- LSTM : 3-(1) 모델을 그대로 사용합니다.

### (2) FastText 모델

-  FastText 모델 학습을 위한 입력 포맷 2차원 리스트 형태 입니다.
  ```
  [['나', '는', '학생', '이다'], ['오늘', '은', '날씨', '가', '좋다']]
  ```

- Word2Vec계열의 FastText를 학습하는 이유
  - n-gram이 추가된 fasttext 모델은 유사한 단어에 대한 임베딩을 word2vec보다 잘 해결할 수 있으며, 오탈자 등에 대한 임베딩 처리가 가능하다.
  - 예) 체크카드, 쳌카드는 word2vec에서는 전혀 다른 단어이지만 fasttext는 character n-gram으로 비교적 같은 단어로 처리할 수 있다.
- 참고: https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText


#### 1) 데이터 준비
* 시작데이터 : clean_train_questions, clean_test_questions

* FastText를 위한 입력 데이터 구조 만들기

#### 2) FastText 모델 생성
* FastText 문법
    * FastText( input데이터,  min_count = , size= , window=  )
        * input데이터 : 학습에 사용할 문장으로 이루어진 리스트
        * min_count : 모델에 사용할 단어의 최소 빈도수. 이 값보다 적게 출현한 단어는 모델에 포함되지 않음. 기본값 = 5
        * size : 단어의 벡터 차원 지정. 기본값 = 100
        * window : 학습할 때 한 단어의 좌우 몇 개의 단어를 보고 예측을 할 것인지를 지정. 기본값 = 5
    * 참조 : https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText

In [ ]:
from gensim.models.fasttext import FastText
import gensim.models.word2vec



#### 3) train에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

### (3) 챗봇 구축
- input 질문
- intent classifier로 common와 faq 중 하나를 예측
- 예측된 intent에 속한 train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
- 코사인 유사도가 가장 높은 top-3개의 Q를 선택
- 선택한 Q에 맵핑된 답변 중 하나를 선택하고 실제 답변과 비교

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

* 예측을 위한 입력 형태로 변환

* 예측하기

* 질문에 대한 벡터 만들기
    * FestText 모델로 부터 벡터 만들기

* train의 질문 벡터들과 유사도 계산
    * FastText 로 만들 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity






#### 2) 함수로 생성하기

In [ ]:
def get_answer3(question): 







    return 

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 5.질문에 대한 답변 비교해보기

* **세부요구사항**
    * 세가지 챗봇을 생성해 보았습니다. 
    * 질문을 입력하여 답변을 비교해 봅시다. 어떤 챗봇이 좀 더 정확한 답변을 하나요?
